In [43]:
import pandas as pd
import numpy as np
import requests as rq

In [2]:
unemp_county = pd.read_csv('output.csv')
unemp_county.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [7]:
df = pd.read_csv('minwage.csv')
act_min_wage = pd.DataFrame()
for i,j in df.groupby('State'):
    if act_min_wage.empty:
        act_min_wage = j.set_index('Year')[['Low.2018']].rename(columns={'Low.2018':i})
    else:
        act_min_wage = act_min_wage.join(j.set_index('Year')[['Low.2018']].rename(columns={'Low.2018':i}))
#act_min_wage['Texas']
        

In [19]:
act_min_wage = act_min_wage.replace(0,np.NaN).dropna(axis=1)


In [20]:
def get_min_wage(year,state):
    try:
        return act_min_wage.loc[year,state]
    except:
        return np.NaN

In [27]:
%%time
unemp_county['minwage'] = list(map(get_min_wage,unemp_county['Year'],unemp_county['State']))

Wall time: 59.8 s


In [28]:
unemp_county.columns

Index(['Year', 'Month', 'State', 'County', 'Rate', 'minwage'], dtype='object')

In [31]:
unemp_county[['Rate','minwage']].corr()

,Rate,minwage
Rate,1.000000,0.153047
minwage,0.153047,1.000000


In [32]:
unemp_county[['Rate','minwage']].cov()

,Rate,minwage
Rate,9.687873,0.651586
minwage,0.651586,1.874228


In [41]:
pres16 = pd.read_csv('pres16results.csv')
pres16['st'].unique()

array(['US', 'CA', 'FL', 'TX', 'NY', 'PA', 'IL', 'OH', 'MI', 'NC', 'GA',
       'VA', 'NJ', 'MA', 'WI', 'MN', 'MO', 'WA', 'IN', 'CO', 'TN', 'MD',
       'SC', 'AL', 'AZ', 'LA', 'KY', 'OR', 'CT', 'IA', 'OK', 'MS', 'KS',
       'NV', 'AR', 'UT', 'NE', 'NM', 'ME', 'NH', 'WV', 'ID', 'MT', 'RI',
       'DE', 'HI', 'SD', 'ND', 'VT', 'DC', 'WY', 'AK', nan], dtype=object)

In [55]:
county2015 = unemp_county.copy()[(unemp_county['Year']==2015) & (unemp_county['Month']=='February')]

In [48]:
web = rq.get("https://www.infoplease.com/state-abbreviations-and-state-postal-codes")
dfs = pd.read_html(web.text)
states_abbv = dfs[0]
abbv_dict = states_abbv.set_index('State/District')[['Postal Code']].to_dict()
abbv_dict = abbv_dict['Postal Code']

In [56]:
county2015['State'] = county2015['State'].map(abbv_dict)

In [57]:
pres16.rename(columns={'st':'State','county':'County'},inplace=True)

In [65]:
for df in [county2015,pres16]:
    df.set_index(['State','County'],inplace=True)

In [66]:
pres16 = pres16[pres16['cand']=='Donald Trump']
pres16 = pres16[['pct']]
pres16.dropna(inplace=True)
pres16

,,pct
State,County,
US,NaN,0.472993
CA,NaN,0.330641
FL,NaN,0.490640
TX,NaN,0.525830
NY,NaN,0.374752
...,...,...
ID,Clark County,0.717314
NE,Arthur County,0.901141
TX,Kenedy County,0.451613


In [80]:
all_together = pres16.merge(county2015,on=['State','County'])
all_together.drop(columns=['Year','Month'],inplace=True)
all_together

,,pct,Rate,minwage
State,County,,,
CA,Los Angeles County,0.234454,7.4,9.51
IL,Cook County,0.214193,6.7,NaN
TX,Harris County,0.418271,4.4,NaN
AZ,Maricopa County,0.491262,5.3,NaN
WA,King County,0.217164,4.4,10.00
...,...,...,...,...
ID,Clark County,0.717314,4.4,7.66
NE,Arthur County,0.901141,4.4,8.45
TX,Kenedy County,0.451613,2.4,NaN


In [83]:

all_together.cov()

,pct,Rate,minwage
pct,0.023910,-0.070181,-0.076602
Rate,-0.070181,5.362972,0.683870
minwage,-0.076602,0.683870,2.336451
